# Cropped Image Generator

## 필요모듈 임포트

In [1]:
import cv2
import os
import numpy as np
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import re

In [2]:
#-*- encoding: utf8 -*-

## file 불러오기

In [3]:
# img_dir = './data/fashion_data/train/원천데이터'
# json_dir = './data/fashion_data/train/라벨링데이터'
cropped_dir = './data/fashion_data/cropped'

## 파일 이름 생성 규칙

image_{1}\_{1}\_{category_name}\_{세부이름들}
- 첫번째 {} : 스타일
    - {'레트로' : 1, '로맨틱' : 2, '리조트' : 3.... } 이런식
- 두번째 {} : 이미지 분류 넘버
    - 이미지 순서 식별자
- category_name 
    - 옷 대분류
- 세부이름들 
    - 색상, 서브색상, 카테고리, 소매기장, 소재 등등...

### 설명 추가
`image{number}_{style}_{카테고리}_{서브카테고리}_{컬러}_{서브컬러}_{소재}_{디테일}_{프린트}_{기장}_{소매기장}_{넥라인}_{칼라}_{핏}.jpg`

In [ ]:
# 이름(스타일) 항목 리스트
name_ls = ['레트로', '로맨틱', '리조트', '매니시', '모던', '밀리터리', '섹시', '소피스트케이티드', 
           '스트리트', '스포티', '아방가르드', '오리엔탈', '웨스턴', '젠더리스', '컨트리', '클래식', '키치', '톰보이', '펑크', '페미닌', '프레피', '히피', '힙합']

In [3]:
#카테고리 리스트
category_ls = ['카테고리', '색상', '서브색상', '소재', '디테일', '프린트', '기장', '소매기장', '넥라인', '칼라', '핏']

In [4]:
def change_number(ls):
    for i in range(4):
        if ls[i] <= 0:
            ls[i] = 1
    return ls

In [12]:
#-*- encoding: utf8 -*-
category_ls = ['카테고리', '색상', '서브색상', '소재', '디테일', '프린트', '기장', '소매기장', '넥라인', '칼라', '핏']
name_ls = os.listdir(img_dir)
image_number = 1
image_sub_number = 1
for i in name_ls :
    
    # 새 directory 만들기
    new_img_dir = img_dir+'/'+i
    new_json_dir = json_dir+'/'+i
    
    # json 경로에서 json 파일 불러와 읽기
    for fn in tqdm(os.listdir(new_json_dir)):
        with open(os.path.join(new_json_dir, fn), 'r', encoding='utf-8') as f:
            json_data = json.load(f)
        
        # json 파일에서 이름을 추출해 img로 만들기
        path = np.fromfile(os.path.join(new_img_dir, json_data['이미지 정보']['이미지 파일명']), np.uint8)
        img = cv2.imdecode(path, cv2.IMREAD_COLOR)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        
        # boundingbox 좌표 가져오기
        for category in json_data['데이터셋 정보']['데이터셋 상세설명']['렉트좌표'].keys():

            if json_data['데이터셋 정보']['데이터셋 상세설명']['렉트좌표'][category] != [{}] :
                sub_data = list(json_data['데이터셋 정보']['데이터셋 상세설명']['라벨링'][category])[0]
                file_name = []

                for j in category_ls:
                    if j in sub_data:
                        string = str(sub_data[j])
                        if (j == '프린트') | (j == '소재'):
                            string =string.replace("\\xa0","_") &nbsp
                            string = re.sub('\/', '_', string)
                            file_name.append(string)
                        else:
                            string = re.sub('\/', '_', string)
                            file_name.append(string)
                    else :
                        file_name.append('0')
                

                rect = list(json_data['데이터셋 정보']['데이터셋 상세설명']['렉트좌표'][category][0].values())
                rect = change_number(rect)
                x = int(rect[0])
                y = int(rect[1])
                w = int(rect[2])
                h = int(rect[3])

                cropped_img = img_rgb[y: y + h, x: x + w]


                join_name = '_'.join(file_name)
                img_name = f'image{image_sub_number:07d}_{image_number}_{category}_{join_name}.jpg'


                basename = f'{cropped_dir}/{img_name}'

######################################################################
                try :
                    crop = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
                    extension = os.path.splitext(img_name)[1]
                    result, encoded_img = cv2.imencode(extension, crop)
                
                    if result:
                        with open(basename, mode='w+b') as f:
                            encoded_img.tofile(f)
                except :
                    print(basename)
                    print(fn)
                    

        image_sub_number += 1
    image_number += 1

100%|██████████| 1349/1349 [10:17<00:00,  2.19it/s]
